In [1]:
import numerox as nx
from sklearn.metrics import roc_auc_score, roc_curve, confusion_matrix, classification_report, accuracy_score 
from pipe import *
from more_itertools import *
from keras.models import Sequential
from keras.models import load_model

from keras import regularizers
from keras import callbacks
from keras import layers
from keras.losses import binary_crossentropy
from keras.utils import to_categorical
import numpy as np
from scipy.stats import gmean
from sklearn.model_selection import train_test_split
from keras import backend as K
from multiprocessing import Pool
import pandas as pd
import numpy as np
from sklearn import metrics, preprocessing, linear_model
from sklearn.manifold import TSNE
from matplotlib import pyplot as plt
K.tensorflow_backend._get_available_gpus()

/anaconda/envs/py35/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


['/job:localhost/replica:0/task:0/device:GPU:0']

In [2]:
def plot_history(h):
   
    history_dict = h.history
    acc = history_dict['acc']
    val_acc = history_dict['val_acc']   
    loss = history_dict['loss']
    val_loss = history_dict['val_loss']
    
    fig, (ax1, ax2) = plt.subplots(2,1, figsize=(21,21))

    ax1.plot(range(0, len(acc)), acc, 'bo', label='Training')
    ax1.plot(range(0, len(val_acc)), val_acc, 'b', label='Validation')
    ax1.set_title("Accuracy")
    
    ax2.plot(range(0, len(loss)), loss, 'bo', label='Training')
    ax2.plot(range(0, len(val_loss)), val_loss, 'b', label='Validation')    
    ax2.set_title("Loss")
    ax2.legend()
    plt.xlabel('Epoch')
    plt.legend()
    best_epoch = np.argmin(val_loss)
    print("Best epoch/loss:", best_epoch, "@", val_loss[best_epoch])
    

In [3]:
#folder = "numerai_datasets"

#nx.data.TRAIN_FILE = '%s/numerai_training_data.csv' % folder
#nx.data.TOURNAMENT_FILE = '%s/numerai_tournament_data.csv' % folder

data = nx.download('nr.133.zip')

nr.133.zip: 154MB [01:10, 2.83MB/s]                            

In [4]:
data.save('133.hdf')

In [5]:
data = nx.load_data('133.hdf')

In [6]:
training_eras = set(data["train"].era)

In [7]:
X = data["train"]["era7"].x
y = data["train"]["era7"].y["bernie"]
X_val = data["validation"]["era121"].x
y_val = data["validation"]["era121"].y["bernie"]

X, X_val, y, y_val = train_test_split(X, y)

In [8]:
folder = "best"


def _nnet(input_size):
    nnet = Sequential()
    nnet.add(layers.Dense(40, input_shape= (input_size, ), activation="relu"))
    nnet.add(layers.Dropout(0.5))
    nnet.add(layers.Dense(256, activation="relu"))
    nnet.add(layers.Dropout(0.5))
    nnet.add(layers.Dense(128, activation="relu"))
    nnet.add(layers.Dropout(0.5))
    nnet.add(layers.Dense(1, activation="sigmoid"))
    nnet.compile(optimizer='adam',
                  loss='binary_crossentropy',
                  metrics=['acc'])
    return nnet


In [9]:
models = {}
histories = {}
target = "bernie"


for e in training_eras:
    checkpoint = callbacks.ModelCheckpoint(
        '%s/%s.h5' % (target, e), 
        verbose = 1, 
        monitor = 'val_loss',
        save_best_only = True, 
        mode = 'auto')  

    X, X_val, y, y_val = train_test_split(data["train"][e].x, data["train"][e].y[target])
    nnet = _nnet(X.shape[1])
    history = nnet.fit(X, y, validation_data=(X_val, y_val), epochs = 100, batch_size = 512, callbacks=[checkpoint])
    models[e] = nnet
    histories[e] = history
    

Train on 3170 samples, validate on 1057 samples
Epoch 1/100
3170/3170 [==============================] - 4s 1ms/step - loss: 0.7048 - acc: 0.4950 - val_loss: 0.6939 - val_acc: 0.4882

Epoch 00001: val_loss improved from inf to 0.69390, saving model to bernie/era77.h5
Epoch 2/100
3170/3170 [==============================] - 0s 9us/step - loss: 0.6987 - acc: 0.5076 - val_loss: 0.6929 - val_acc: 0.5137

Epoch 00002: val_loss improved from 0.69390 to 0.69287, saving model to bernie/era77.h5
Epoch 3/100
3170/3170 [==============================] - 0s 8us/step - loss: 0.6986 - acc: 0.4927 - val_loss: 0.6926 - val_acc: 0.5203

Epoch 00003: val_loss improved from 0.69287 to 0.69265, saving model to bernie/era77.h5
Epoch 4/100
3170/3170 [==============================] - 0s 8us/step - loss: 0.6941 - acc: 0.5114 - val_loss: 0.6923 - val_acc: 0.5251

Epoch 00004: val_loss improved from 0.69265 to 0.69228, saving model to bernie/era77.h5
Epoch 5/100
3170/3170 [==============================] - 0s 

3170/3170 [==============================] - 0s 9us/step - loss: 0.6864 - acc: 0.5498 - val_loss: 0.6820 - val_acc: 0.5733

Epoch 00039: val_loss improved from 0.68281 to 0.68202, saving model to bernie/era77.h5
Epoch 40/100
3170/3170 [==============================] - 0s 9us/step - loss: 0.6840 - acc: 0.5571 - val_loss: 0.6799 - val_acc: 0.5837

Epoch 00040: val_loss improved from 0.68202 to 0.67991, saving model to bernie/era77.h5
Epoch 41/100
3170/3170 [==============================] - 0s 9us/step - loss: 0.6867 - acc: 0.5470 - val_loss: 0.6826 - val_acc: 0.5497

Epoch 00041: val_loss did not improve from 0.67991
Epoch 42/100
3170/3170 [==============================] - 0s 8us/step - loss: 0.6869 - acc: 0.5508 - val_loss: 0.6799 - val_acc: 0.5752

Epoch 00042: val_loss improved from 0.67991 to 0.67990, saving model to bernie/era77.h5
Epoch 43/100
3170/3170 [==============================] - 0s 8us/step - loss: 0.6895 - acc: 0.5300 - val_loss: 0.6806 - val_acc: 0.5724

Epoch 00043: 

3170/3170 [==============================] - 0s 8us/step - loss: 0.6832 - acc: 0.5536 - val_loss: 0.6755 - val_acc: 0.5724

Epoch 00080: val_loss did not improve from 0.67196
Epoch 81/100
3170/3170 [==============================] - 0s 9us/step - loss: 0.6785 - acc: 0.5609 - val_loss: 0.6739 - val_acc: 0.5866

Epoch 00081: val_loss did not improve from 0.67196
Epoch 82/100
3170/3170 [==============================] - 0s 9us/step - loss: 0.6821 - acc: 0.5530 - val_loss: 0.6730 - val_acc: 0.5790

Epoch 00082: val_loss did not improve from 0.67196
Epoch 83/100
3170/3170 [==============================] - 0s 9us/step - loss: 0.6768 - acc: 0.5719 - val_loss: 0.6733 - val_acc: 0.5771

Epoch 00083: val_loss did not improve from 0.67196
Epoch 84/100
3170/3170 [==============================] - 0s 10us/step - loss: 0.6785 - acc: 0.5640 - val_loss: 0.6760 - val_acc: 0.5667

Epoch 00084: val_loss did not improve from 0.67196
Epoch 85/100
3170/3170 [==============================] - 0s 11us/step -

3273/3273 [==============================] - 0s 8us/step - loss: 0.6895 - acc: 0.5371 - val_loss: 0.6897 - val_acc: 0.5600

Epoch 00021: val_loss did not improve from 0.68751
Epoch 22/100
3273/3273 [==============================] - 0s 8us/step - loss: 0.6894 - acc: 0.5338 - val_loss: 0.6881 - val_acc: 0.5665

Epoch 00022: val_loss did not improve from 0.68751
Epoch 23/100
3273/3273 [==============================] - 0s 8us/step - loss: 0.6886 - acc: 0.5396 - val_loss: 0.6880 - val_acc: 0.5573

Epoch 00023: val_loss did not improve from 0.68751
Epoch 24/100
3273/3273 [==============================] - 0s 8us/step - loss: 0.6879 - acc: 0.5451 - val_loss: 0.6885 - val_acc: 0.5509

Epoch 00024: val_loss did not improve from 0.68751
Epoch 25/100
3273/3273 [==============================] - 0s 8us/step - loss: 0.6865 - acc: 0.5518 - val_loss: 0.6856 - val_acc: 0.5426

Epoch 00025: val_loss improved from 0.68751 to 0.68563, saving model to bernie/era92.h5
Epoch 26/100
3273/3273 [============

Epoch 62/100
3273/3273 [==============================] - 0s 8us/step - loss: 0.6771 - acc: 0.5695 - val_loss: 0.6729 - val_acc: 0.5747

Epoch 00062: val_loss did not improve from 0.66597
Epoch 63/100
3273/3273 [==============================] - 0s 8us/step - loss: 0.6780 - acc: 0.5658 - val_loss: 0.6739 - val_acc: 0.5765

Epoch 00063: val_loss did not improve from 0.66597
Epoch 64/100
3273/3273 [==============================] - 0s 8us/step - loss: 0.6763 - acc: 0.5765 - val_loss: 0.6676 - val_acc: 0.5738

Epoch 00064: val_loss did not improve from 0.66597
Epoch 65/100
3273/3273 [==============================] - 0s 8us/step - loss: 0.6715 - acc: 0.5796 - val_loss: 0.6703 - val_acc: 0.5775

Epoch 00065: val_loss did not improve from 0.66597
Epoch 66/100
3273/3273 [==============================] - 0s 8us/step - loss: 0.6742 - acc: 0.5863 - val_loss: 0.6666 - val_acc: 0.5784

Epoch 00066: val_loss did not improve from 0.66597
Epoch 67/100
3273/3273 [==============================] - 0s

...

3543/3543 [==============================] - 0s 13us/step - loss: 0.6919 - acc: 0.5185 - val_loss: 0.6927 - val_acc: 0.5064

Epoch 00011: val_loss did not improve from 0.69224
Epoch 12/100
3543/3543 [==============================] - 0s 13us/step - loss: 0.6908 - acc: 0.5272 - val_loss: 0.6929 - val_acc: 0.5055

Epoch 00012: val_loss did not improve from 0.69224
Epoch 13/100
3543/3543 [==============================] - 0s 13us/step - loss: 0.6932 - acc: 0.5134 - val_loss: 0.6924 - val_acc: 0.5080

Epoch 00013: val_loss did not improve from 0.69224
Epoch 14/100
3543/3543 [==============================] - 0s 13us/step - loss: 0.6907 - acc: 0.5363 - val_loss: 0.6926 - val_acc: 0.5055

Epoch 00014: val_loss did not improve from 0.69224
Epoch 15/100
3543/3543 [==============================] - 0s 12us/step - loss: 0.6907 - acc: 0.5295 - val_loss: 0.6923 - val_acc: 0.5174

Epoch 00015: val_loss did not improve from 0.69224
Epoch 16/100
3543/3543 [==============================] - 0s 12us/st

3543/3543 [==============================] - 0s 12us/step - loss: 0.6847 - acc: 0.5546 - val_loss: 0.6863 - val_acc: 0.5445

Epoch 00052: val_loss did not improve from 0.68496
Epoch 53/100
3543/3543 [==============================] - 0s 12us/step - loss: 0.6843 - acc: 0.5594 - val_loss: 0.6905 - val_acc: 0.5233

Epoch 00053: val_loss did not improve from 0.68496
Epoch 54/100
3543/3543 [==============================] - 0s 12us/step - loss: 0.6819 - acc: 0.5586 - val_loss: 0.6867 - val_acc: 0.5368

Epoch 00054: val_loss did not improve from 0.68496
Epoch 55/100
3543/3543 [==============================] - 0s 12us/step - loss: 0.6856 - acc: 0.5461 - val_loss: 0.6862 - val_acc: 0.5470

Epoch 00055: val_loss did not improve from 0.68496
Epoch 56/100
3543/3543 [==============================] - 0s 12us/step - loss: 0.6853 - acc: 0.5470 - val_loss: 0.6850 - val_acc: 0.5546

Epoch 00056: val_loss did not improve from 0.68496
Epoch 57/100
3543/3543 [==============================] - 0s 11us/st

3543/3543 [==============================] - 0s 12us/step - loss: 0.6811 - acc: 0.5622 - val_loss: 0.6883 - val_acc: 0.5360

Epoch 00095: val_loss did not improve from 0.68371
Epoch 96/100
3543/3543 [==============================] - 0s 12us/step - loss: 0.6816 - acc: 0.5588 - val_loss: 0.6841 - val_acc: 0.5436

Epoch 00096: val_loss did not improve from 0.68371
Epoch 97/100
3543/3543 [==============================] - 0s 13us/step - loss: 0.6823 - acc: 0.5611 - val_loss: 0.6845 - val_acc: 0.5521

Epoch 00097: val_loss did not improve from 0.68371
Epoch 98/100
3543/3543 [==============================] - 0s 12us/step - loss: 0.6797 - acc: 0.5690 - val_loss: 0.6862 - val_acc: 0.5453

Epoch 00098: val_loss did not improve from 0.68371
Epoch 99/100
3543/3543 [==============================] - 0s 12us/step - loss: 0.6801 - acc: 0.5715 - val_loss: 0.6843 - val_acc: 0.5402

Epoch 00099: val_loss did not improve from 0.68371
Epoch 100/100
3543/3543 [==============================] - 0s 12us/s

3414/3414 [==============================] - 0s 12us/step - loss: 0.6468 - acc: 0.6248 - val_loss: 0.6434 - val_acc: 0.6274

Epoch 00033: val_loss improved from 0.64410 to 0.64340, saving model to bernie/era105.h5
Epoch 34/100
3414/3414 [==============================] - 0s 12us/step - loss: 0.6454 - acc: 0.6350 - val_loss: 0.6459 - val_acc: 0.6248

Epoch 00034: val_loss did not improve from 0.64340
Epoch 35/100
3414/3414 [==============================] - 0s 12us/step - loss: 0.6469 - acc: 0.6277 - val_loss: 0.6438 - val_acc: 0.6309

Epoch 00035: val_loss did not improve from 0.64340
Epoch 36/100
3414/3414 [==============================] - 0s 12us/step - loss: 0.6414 - acc: 0.6242 - val_loss: 0.6466 - val_acc: 0.6221

Epoch 00036: val_loss did not improve from 0.64340
Epoch 37/100
3414/3414 [==============================] - 0s 12us/step - loss: 0.6453 - acc: 0.6254 - val_loss: 0.6448 - val_acc: 0.6274

Epoch 00037: val_loss did not improve from 0.64340
Epoch 38/100
3414/3414 [======

3414/3414 [==============================] - 0s 13us/step - loss: 0.6214 - acc: 0.6591 - val_loss: 0.6308 - val_acc: 0.6529

Epoch 00074: val_loss did not improve from 0.62943
Epoch 75/100
3414/3414 [==============================] - 0s 13us/step - loss: 0.6264 - acc: 0.6602 - val_loss: 0.6321 - val_acc: 0.6485

Epoch 00075: val_loss did not improve from 0.62943
Epoch 76/100
3414/3414 [==============================] - 0s 13us/step - loss: 0.6175 - acc: 0.6690 - val_loss: 0.6300 - val_acc: 0.6467

Epoch 00076: val_loss did not improve from 0.62943
Epoch 77/100
3414/3414 [==============================] - 0s 13us/step - loss: 0.6201 - acc: 0.6558 - val_loss: 0.6281 - val_acc: 0.6564

Epoch 00077: val_loss improved from 0.62943 to 0.62806, saving model to bernie/era105.h5
Epoch 78/100
3414/3414 [==============================] - 0s 12us/step - loss: 0.6203 - acc: 0.6629 - val_loss: 0.6275 - val_acc: 0.6503

Epoch 00078: val_loss improved from 0.62806 to 0.62749, saving model to bernie/er

In [6]:
models = {}
target = "bernie"
for e in training_eras:
    models[e] = load_model("%s/%s.h5" % (target, e))

In [10]:
res = np.arange(0, 1, 0.01)

hists = list(training_eras | select(lambda e: data["train"][e].x) | select(lambda d: np.histogram(list(flatten(d)), bins=res)[0]))

In [11]:
def hist_overlap(a, b):
    mins = np.minimum(a, b)
    return np.true_divide(np.sum(mins), np.sum(b))

def predict_era(d):
    d_hist = np.histogram(list(flatten(d)), bins=res)[0]
    predicted_era = np.argmax(np.array(hists | select(lambda h: hist_overlap(h, d_hist))))
    return "era%d" % predicted_era + 1
                              

In [ ]:

from multiprocessing import Pool

hist_data_stats = {}

def work(hist_data):
    h1, x_data, y = hist_data
    h2 = np.histogram(list(flatten(x_data)), bins=res)[0]
    overlap = hist_overlap(h1, h2)
    preds = models[e].predict(x_data).reshape((len(x_data), ))
    loss = metrics.log_loss(y, preds)
    return (loss, overlap)

n = 0

with Pool() as pool:
    for e in training_eras:
        n += 1
        h1 = np.histogram(list(flatten(data["train"][e].x)), bins=res)[0]
        comparison_data = ((h1, data["train"][e2].x, data["train"][e2].y[target]) for e2 in training_eras if e2 != e)

        ol_loss = pool.map(work, comparison_data)

        hist_data_stats[e] = ol_loss
        
        print(n, "/", len(training_eras), "@", e, "processed. ", len(ol_loss), "added")

In [ ]:
losses = list(hist_data_stats.values() | select(lambda h: h[0]))
overlaps = list(hist_data_stats.values() | select(lambda h: h[1]))

In [ ]:
plt.figure(figsize=(21,16))
plt.scatter(overlaps, losses)


In [ ]:
np.array(overlaps).save("overlaps.pkl")
np.array(losses).save("losses.pkl")